In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
spark = SparkSession.builder.appName("ReadCSV").getOrCreate()


24/06/26 18:59:34 WARN Utils: Your hostname, umar-ThinkPad-T470 resolves to a loopback address: 127.0.1.1; using 192.168.18.21 instead (on interface wlp4s0)
24/06/26 18:59:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/26 18:59:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
csv_file_path = 'accidents/US_Accidents_March23.csv'

# Read the CSV file into a Spark DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df.show()




24/06/26 19:00:30 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+-------+--------+-------------------+-------------------+------------------+------------------+-------+-------+------------+--------------------+--------------------+------------+----------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
|  ID| Source|Severity|         Start_Time|           End_Time|         Start_Lat|         Start_Lng|End_Lat|End_Lng|Distance(mi)|         Description|              Street|        City|    County|State|   Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|A

In [4]:
row_count = df.count()

# Count the number of columns
column_count = len(df.columns)

# Print the counts
print(f"Number of rows: {row_count}")
print(f"Number of columns: {column_count}")

Number of rows: 7728394
Number of columns: 46


In [6]:
df=df.dropna()

In [7]:
row_count = df.count()

# Count the number of columns
column_count = len(df.columns)

# Print the counts
print(f"Number of rows: {row_count}")
print(f"Number of columns: {column_count}")

Number of rows: 3554549
Number of columns: 46


In [8]:
# Select relevant columns for analysis
columns_of_interest = ['Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng', 'Weather_Condition', 'Visibility(mi)', 'Precipitation(in)', 'Severity', 'City', 'State']
df_filtered = df.select(columns_of_interest)

# Extract hour from Start_Time for time of day analysis
df_filtered = df_filtered.withColumn('Hour', F.hour('Start_Time'))

# Aggregate data to find the number of accidents by hour, weather condition, and road condition
accidents_by_hour = df_filtered.groupBy('Hour').count().orderBy('Hour')
accidents_by_weather = df_filtered.groupBy('Weather_Condition').count().orderBy(F.desc('count'))

# Aggregate data to find accident hotspots (e.g., by city or by geographical location)
accidents_by_city = df_filtered.groupBy('City', 'State').count().orderBy(F.desc('count'))

In [9]:
# Convert to Pandas DataFrame
accidents_by_hour_pd = accidents_by_hour.toPandas()
accidents_by_weather_pd = accidents_by_weather.toPandas()
accidents_by_city_pd = accidents_by_city.limit(100).toPandas()  # Limit to top 100 cities for visualization


In [11]:
import plotly.express as px

# Plot accidents by hour
fig_hour = px.bar(accidents_by_hour_pd, x='Hour', y='count', title='Accidents by Hour of Day')
fig_hour.show()

# Plot accidents by weather condition
fig_weather = px.bar(accidents_by_weather_pd, x='Weather_Condition', y='count', title='Accidents by Weather Condition')
fig_weather.show()

# Plot accident hotspots by city
fig_city = px.bar(accidents_by_city_pd, x='City', y='count', title='Accident Hotspots by City', labels={'count': 'Number of Accidents'})
fig_city.show()




In [14]:
accidents_by_city = df_filtered.groupBy('City', 'State', 'Start_Lat', 'Start_Lng').count().orderBy(F.desc('count'))

In [15]:
# Convert to Pandas DataFrame
accidents_by_city_pd = accidents_by_city.limit(1000).toPandas()  # Limit to top 1000 locations for visualization


In [16]:
# For geographical visualization of hotspots
fig_geo = px.scatter_mapbox(
    accidents_by_city_pd, lat='Start_Lat', lon='Start_Lng', size='count', 
    color='count', hover_name='City', hover_data=['State'],
    title='Accident Hotspots',
    mapbox_style='carto-positron', zoom=3, height=600
)
fig_geo.show()

In [ ]:
spark.stop()